In [ ]:
# DATASETS
# DOWNLOAD COCO
!bash scripts/coco.sh

# INDIVIDUAL FRAMES MUST BE STORED IN ./dataset/frames/train/
# game frames stored in trainA and movie frames stored in trainB


In [ ]:
# TRAIN INSTANCE SEGMENTATION MODEL

from mmcv import Config
from mmdet.apis import train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.utils import update_data_root

cfg = Config.fromfile("./detectoRS/configs/custom/detectors_instaboost.py")

update_data_root(cfg)
model = build_detector(cfg.model)

datasets = [build_dataset(cfg.data.train)]

model.CLASSES = cfg.CLASSES
train_detector(
        model,
        datasets,
        cfg,
        distributed=False,
        validate=True,
)

In [ ]:
# TEST INSTANCE SEGMENTATION MODEL

from mmcv import Config
from mmdet.apis import train_detector,  single_gpu_test
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.models import build_detector
from mmdet.utils import update_data_root
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

cfg = Config.fromfile("./detectoRS/configs/custom/detectors_instaboost.py")

# update data root according to MMDET_DATASETS
update_data_root(cfg)

# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    shuffle=False,
    samples_per_gpu=1,
    workers_per_gpu=1
)

# build the model and load checkpoint
cfg.model.train_cfg = None

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, './detectoRS/checkpoints/detectors_finetuned.pth', map_location='cpu')
    
model.CLASSES = cfg.CLASSES

model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader, None, None, 0.3)

dataset.format_results(outputs, **{'jsonfile_prefix': '../dataset/annotation/test'})


In [ ]:
# TRAIN CLASSIFICATION MODEL

from mmcv import Config
from mmcls.apis import train_model
from mmcls.datasets import build_dataset
from mmcls.models import build_classifier

cfg = Config.fromfile('./swin_transformer/configs/custom/swin_tiny.py')
  
model = build_classifier(cfg.model)
model.init_weights()

datasets = [build_dataset(cfg.data.train)]

train_model(
    model,
    datasets,
    cfg
)

In [ ]:
# TEST CLASSIFICATION MODEL

import mmcv
import numpy as np
from mmcv.parallel import MMDataParallel
from mmcv.runner import load_checkpoint

from mmcls.apis import single_gpu_test
from mmcls.datasets import build_dataloader, build_dataset
from mmcls.models import build_classifier

cfg = mmcv.Config.fromfile('./swin_transformer/configs/custom/swin_tiny.py')

dataset = build_dataset(cfg.data.test, default_args=dict(test_mode=True))
data_loader = build_dataloader(
    dataset,
    shuffle=False,
    samples_per_gpu=1,
    workers_per_gpu=1
)

model = build_classifier(cfg.model)
checkpoint = load_checkpoint(model, './swin_transformer/checkpoints/swin_finetuned.pth', map_location='cpu')

if 'CLASSES' in checkpoint.get('meta', {}):
    CLASSES = checkpoint['meta']['CLASSES']

outputs = single_gpu_test(model, data_loader, None, None)

results = {}
scores = np.vstack(outputs)
pred_score = np.max(scores, axis=1)
pred_label = np.argmax(scores, axis=1)
pred_class = [CLASSES[lb] for lb in pred_label]
res_items = {
    'class_scores': scores,
    'pred_score': pred_score,
    'pred_label': pred_label,
    'pred_class': pred_class
}
results.update(res_items)
        
mmcv.dump(results, "./classification_results.json")

In [ ]:
# TRAIN TRANSLATION MODEL

# AtoB or BtoA
direction = "AtoB"

# 2.2) mask or no-mask
mask = "no-mask"

# 2.3) sample or no-sample
sample = "sample"

# num epochs
epochs = 50

!bash cut/train.sh {direction} {mask} {sample} {epochs}

In [ ]:
# TEST TRANSLATION MODEL

# AtoB or BtoA
direction = "AtoB"

# 2.2) mask or no-mask
mask = "no-mask"

# 2.3) sample or no-sample
sample = "sample"

!bash cut/test.sh {direction} {mask} {sample}


In [ ]:
# CALCULATE FID SCORE

# AtoB or BtoA
direction = "AtoB"

# 2.2) mask or no-mask
mask = "no-mask"

# 2.3) sample or no-sample
sample = "sample"

!bash cut/fid.sh 